In [79]:
# Dependencies
import pymongo
import pandas as pd

In [80]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.Olympic_data

In [81]:
df = pd.read_csv("Cleaned.csv")
df = df[["ID", "Region", "Name", "Sex", "Year", "Sport", "Event", "Bronze", "Silver", "Gold", "No Win", "Attempts", "Wins"]]
df["Total Medals"] = df["Bronze"] + df["Gold"] + df["Silver"]
df.head(5)

,ID,Region,Name,Sex,Year,Sport,Event,Bronze,Silver,Gold,No Win,Attempts,Wins,Total Medals
0,1,China,A Dijiang,M,1992,Basketball,Basketball Men's Basketball,0,0,0,1,1,0,0
1,2,China,A Lamusi,M,2012,Judo,Judo Men's Extra-Lightweight,0,0,0,1,1,0,0
2,602,China,Abudoureheman,M,2000,Boxing,Boxing Men's Middleweight,0,0,0,1,1,0,0
3,1463,China,Ai Linuer,M,2004,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",0,0,0,1,1,0,0
4,1464,China,Ai Yanhan,F,2016,Swimming,Swimming Women's 200 metres Freestyle,0,0,0,1,1,0,0


In [93]:
#Gender Over Time CSV
# Sport,Year,Sex,Athletes,Bronze,Silver,Gold,No Win,Attempts,Wins
GenderOverTime = df.groupby(["Sport","Year","Sex",]).agg({
    "Name":"nunique",
    "Bronze": "sum",
    "Silver":"sum",
    "Gold":"sum",
    "No Win": "sum",
    "Attempts":"sum",
    "Wins":"sum"
}).reset_index()

,Sport,Year,Sex,Name,Bronze,Silver,Gold,No Win,Attempts,Wins
0,Aeronautics,1936,M,1,0,0,1,0,1,1
1,Alpine Skiing,1936,F,37,1,1,1,34,37,3
2,Alpine Skiing,1936,M,66,1,1,1,63,66,3
3,Alpine Skiing,1948,F,40,3,3,3,84,93,9
4,Alpine Skiing,1948,M,127,4,3,3,257,267,10


In [83]:
#MapData CSV
MapData = df.groupby(["Region"]).agg({
    "Bronze": "sum",
    "Silver":"sum",
    "Gold":"sum",
    "No Win": "sum",
    "Attempts":"sum",
    "Wins":"sum"
}).reset_index()

In [84]:
#Map Over time CSV
MedalsOverTime = df.groupby(["Region","Year"]).agg({
    "Bronze": "sum",
    "Silver":"sum",
    "Gold":"sum",
    "No Win": "sum",
    "Attempts":"sum",
    "Wins":"sum"
}).reset_index()

In [85]:
#Bar Chart CSV
##top 10 countries
countries = df.groupby(["Region"]).agg({
    "Gold":"sum",
    "Total Medals":"sum"
}).reset_index().sort_values(by="Gold", ascending=False).head(10)
countries["Category"] = "Country"
countries = countries.rename(columns={"Region":"Item"})
##top 10 athletes
athletes = df.groupby(["Name"]).agg({
    "Gold":"sum",
    "Total Medals":"sum"
}).reset_index().sort_values(by="Gold", ascending=False).head(10)
athletes["Category"] = "Athlete"
athletes = athletes.rename(columns={"Name":"Item"})
##top 10 sports
sports = df.groupby(["Sport"]).agg({
    "Gold":"sum",
    "Total Medals":"sum"
}).reset_index().sort_values(by="Gold", ascending=False).head(10)
sports["Category"] = "Sport"
sports = sports.rename(columns={"Sport":"Item"})
##top 10 events
events = df.groupby(["Event"]).agg({
    "Gold":"sum",
    "Total Medals":"sum"
}).reset_index().sort_values(by="Gold", ascending=False).head(10)
events["Category"] = "Events"
events = events.rename(columns={"Event":"Item"})

Barchart = countries.append([athletes,events,sports],verify_integrity=False).set_index("Category").reset_index()

In [86]:
#Port to Mongo DB
All_Data_db = db.AllData_DB
GenderOverrTime_db = db.GenderOverTime_DB
MapData_db = db.MapData_DB
MedalsOverTime_db = db.MedalsOverTime_DB
Barchart_db = db.Barchart

In [87]:
All_Data_db.insert_many(df.to_dict("records"))

In [94]:
GenderOverrTime_db.insert_many(GenderOverTime.to_dict("records"))

In [90]:
MapData_db.insert_many(MapData.to_dict("records"))

In [89]:
MedalsOverTime_db.insert_many(MedalsOverTime.to_dict("records"))

In [88]:
Barchart_db.insert_many(Barchart.to_dict("records"))